In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [3]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [4]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [5]:
def RF_Reg(space):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized
    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe UV Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
    y = df['Peak_eV']
    z = df['FWHM_eV']
    group = df['Expt. No']

    regressor_peak = RandomForestRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                           max_samples = 0.5,
                                           n_estimators = 100,
                                           random_state=0)

    regressor_fwhm = RandomForestRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                           max_samples = 0.5,
                                           n_estimators = 100,
                                           random_state=0)

    regressor_cascade = RandomForestRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                           max_samples = 0.5,
                                           n_estimators = 100,
                                           random_state=0)

    gkf = GroupKFold(n_splits=10)
    matrix = np.zeros([10,13])
    matrix_fwhm = np.zeros([10,13])
    matrix_cascade = np.zeros([10,14])
    for i in range(0,1):
        j = 0
        for train_index, test_index in gkf.split(X, groups=group):
            #Separate out training and test sets
            X_train = X.iloc[train_index, :]
            X2_train = X2.iloc[train_index, :]
            y_train = y[train_index]
            z_train = z[train_index]
            X_test = X.iloc[test_index, :]
            y_test = y[test_index]
            z_test = z[test_index]

            #prediction for Peak
            regressor_peak.fit(X_train, y_train)
            y_pred = regressor_peak.predict(X_test)
            #prediction for FWHM (direct)
            regressor_fwhm.fit(X_train, z_train)
            y_pred_fwhm = regressor_fwhm.predict(X_test)
            #prediction for FWHM (cascade)
            regressor_cascade.fit(X2_train, z_train)
            X_test.insert(5, "Peak_eV", regressor_peak.predict(X_test), True)
            y_pred_cascade = regressor_cascade.predict(X_test)

            #errors in eV
            matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
            matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
            matrix[10*i + j,2] = rae(y_test, y_pred)
            matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

            matrix_fwhm[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,1] = metrics.r2_score(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,2] = rae(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))

            matrix_cascade[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,1] = metrics.r2_score(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,2] = rae(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))

            #errors in nm
            #UNnormalize the data first
            y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            z_test = z_test*(0.35)+0.05
            y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
            y_pred_cascade = y_pred_cascade*(0.35)+0.05
            matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
            matrix[10*i + j,8:13] = regressor_peak.feature_importances_

            matrix_fwhm[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
            matrix_fwhm[10*i + j,8:13] = regressor_fwhm.feature_importances_

            matrix_cascade[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
            matrix_cascade[10*i + j,8:14] = regressor_cascade.feature_importances_

            j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = matrix.mean(0)[:8]
    matrix_2[0,8:16] = np.std(matrix[:,:8], axis=0)
    matrix_2[0,16:21] = matrix.mean(0)[8:13]

    matrix_3 = np.zeros([1,21])
    matrix_3[0,:8] = matrix_fwhm.mean(0)[:8]
    matrix_3[0,8:16] = np.std(matrix_fwhm[:,:8], axis=0)
    matrix_3[0,16:21] = matrix_fwhm.mean(0)[8:13]

    matrix_4 = np.zeros([1,22])
    matrix_4[0,:8] = matrix_cascade.mean(0)[:8]
    matrix_4[0,8:16] = np.std(matrix_cascade[:,:8], axis=0)
    matrix_4[0,16:22] = matrix_cascade.mean(0)[8:14]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0], matrix_2[0,8])
    print("R2 (norm, eV):", matrix_2[0,1], matrix_2[0,9])
    print("RAE (norm, eV):", matrix_2[0,2], matrix_2[0,10])
    print("RMSE (norm, eV):", matrix_2[0,3], matrix_2[0,11])
    print("Importances", matrix_2[0,8:13])
    print("MAE (nm):", matrix_2[0,4], matrix_2[0,12])
    print("R2 (nm):", matrix_2[0,5], matrix_2[0,13])
    print("RAE (nm):", matrix_2[0,6], matrix_2[0,14])
    print("RMSE (nm):", matrix_2[0,7], matrix_2[0,15])

    print('\033[1m' + 'Absorption FWHM (direct)' + '\033[0m')
    print("MAE (norm, eV):", matrix_3[0,0], matrix_3[0,8])
    print("R2 (norm, eV):", matrix_3[0,1], matrix_3[0,9])
    print("RAE (norm, eV):", matrix_3[0,2], matrix_3[0,10])
    print("RMSE (norm, eV):", matrix_3[0,3], matrix_3[0,11])
    print("Importances", matrix_3[0,8:13])
    print("MAE (nm):", matrix_3[0,4], matrix_3[0,12])
    print("R2 (nm):", matrix_3[0,5], matrix_3[0,13])
    print("RAE (nm):", matrix_3[0,6], matrix_3[0,14])
    print("RMSE (nm):", matrix_3[0,7], matrix_3[0,15])

    print('\033[1m' + 'Absorption FWHM (cascade)' + '\033[0m')
    print("MAE (norm, eV):", matrix_4[0,0], matrix_4[0,8])
    print("R2 (norm, eV):", matrix_4[0,1], matrix_4[0,9])
    print("RAE (norm, eV):", matrix_4[0,2], matrix_4[0,10])
    print("RMSE (norm, eV):", matrix_4[0,3], matrix_4[0,11])
    print("Importances", matrix_4[0,8:14])
    print("MAE (nm):", matrix_4[0,4], matrix_4[0,12])
    print("R2 (nm):", matrix_4[0,5], matrix_4[0,13])
    print("RAE (nm):", matrix_4[0,6], matrix_4[0,14])
    print("RMSE (nm):", matrix_4[0,7], matrix_4[0,15])

    return {'loss': matrix_3[0,3], 'status': STATUS_OK }

In [6]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

space = { 'x1': hp.quniform('x1', 10,20,1),
            'x2': hp.quniform('x2', 2,20,1),
            'x3': hp.quniform('x3', 1,10,1),
            'x4': hp.quniform('x4', 1,5,1),
        }

best = fmin(RF_Reg,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            rstate=rstate)

print(best)

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.046676148229547666                                                                                                   
0.010504788470197741                                                                                                   
R2 (norm, eV):                                                                                                         
0.8476407889484443                                                                                                     
0.0718672388736606                                                                                                     
RAE (norm, eV):                                                                                                        
0.3378599090471474                              

[0.00435288 0.12285083 0.05678873 0.00885485 0.45490656 0.16137919]                                                    
MAE (nm):                                                                                                              
2.539362597622664                                                                                                      
0.4549065550813917                                                                                                     
R2 (nm):                                                                                                               
0.5563648181445304                                                                                                     
0.16137919180368385                                                                                                    
RAE (nm):                                                                                                              
0.49539357952225205                     

MAE (norm, eV):                                                                                                        
0.031102450582178375                                                                                                   
0.004874573245074647                                                                                                   
R2 (norm, eV):                                                                                                         
0.5356866195690123                                                                                                     
0.12121722022763745                                                                                                    
RAE (norm, eV):                                                                                                        
0.5573945650912948                                                                                                     
0.06382884403474115                     

MAE (nm):                                                                                                              
2.7632510648020006                                                                                                     
0.4511319705103149                                                                                                     
R2 (nm):                                                                                                               
0.5560948705719058                                                                                                     
0.13528859355369363                                                                                                    
RAE (nm):                                                                                                              
0.5382797446119258                                                                                                     
0.08068071257652619                     

0.029516449395490335                                                                                                   
0.004795568681113811                                                                                                   
R2 (norm, eV):                                                                                                         
0.58621158191364                                                                                                       
0.11542281006386873                                                                                                    
RAE (norm, eV):                                                                                                        
0.5295459037939441                                                                                                     
0.0713072260289469                                                                                                     
RMSE (norm, eV):                        

...

0.06263612521768923                                                                                                    
RMSE (norm, eV):                                                                                                       
0.044364302774633164                                                                                                   
0.008623414836428676                                                                                                   
Importances                                                                                                            
[0.00429034 0.10602474 0.06263613 0.00862341 0.45742434]                                                               
MAE (nm):                                                                                                              
2.547877468877478                                                                                                      
0.4574243381357825                      

0.0811927485401139                                                                                                     
RMSE (nm):                                                                                                             
11.12044898925122                                                                                                      
2.671904611065676                                                                                                      
Absorption FWHM (direct)                                                                                       
MAE (norm, eV):                                                                                                        
0.028617296891845435                                                                                                   
0.0045416820882057335                                                                                                  
R2 (norm, eV):                                  

RMSE (norm, eV):                                                                                                       
0.06319372498161108                                                                                                    
0.012899384270371565                                                                                                   
Importances                                                                                                            
[0.0107059  0.07093285 0.08613537 0.01289938 2.29681202]                                                               
MAE (nm):                                                                                                              
9.852013809641255                                                                                                      
2.296812020448914                                                                                                      
R2 (nm):                                

RMSE (nm):                                                                                                             
4.295651074055289                                                                                                      
1.045891542835526                                                                                                      
Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.04534241540983589                                                                                                    
0.009164841964969904                                                                                                   
R2 (norm, eV):                                                                                                         
0.8644913847634378                              

0.04809086789154754                                                                                                    
0.009091112390553683                                                                                                   
Importances                                                                                                            
[0.00514715 0.12791966 0.07258583 0.00909111 0.47026222 0.16377087]                                                    
MAE (nm):                                                                                                              
2.7017132055447712                                                                                                     
0.4702622203205412                                                                                                     
R2 (nm):                                                                                                               
0.5505048747911154                      